In [46]:
import cv2
import numpy as np

#1
roi = None
drag_start = None
mouse_status = 0
tracking_start = False
# 4개 변수 초기화, 마우스 이벤트에 사용

# mouse event 함수 정의
def onMouse(event, x, y, flags, param=None):
    global roi
    global drag_start
    global mouse_status
    global tracking_start
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drag_start = (x, y) # 마우스 드래그 포인트
        mouse_status = 1  # 1은 roi를 설정하기 시작했다는 의미
        tracking_start = False # 아직 트랙킹 안 한 상태
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            xmin = min(x, drag_start[0])
            ymin = min(y, drag_start[1])
            xmax = max(x, drag_start[0])
            ymax = max(y, drag_start[1])
            roi = (xmin, ymin, xmax, ymax)
            mouse_status = 2 # dragging
            # 마우스 시작포인트, 끝 포인트 즉 roi 영역 저장, 드래그 하는 중이라는 의미
            
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_status = 3 # complete 마우스에서 손을 떼었다라는 의미

In [47]:

cv2.namedWindow('tracking') # tracking 이라는 윈도우 생성
cv2.setMouseCallback('tracking', onMouse) # onMous에 등록


cap = cv2.VideoCapture('/Users/iseung-won/images/lucas2.mp4') # 새로운 영상 읽기
if (not cap.isOpened()):
    print('Error opening video')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), 
                 int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
roi_mask = np.zeros((height, width), dtype=np.uint8) 
# 동영상과 같은 사이즈의 roi 마스크 생성. black

params = dict(maxCorners=16, qualityLevel=0.1, minDistance=1, blockSize=5)
# 밑에 goodFeaturesToTrack에 사용
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
# 계속 돌아가지 않게 종료 조건 설정 시 사용, 10번
params2 = dict(winSize=(5,5), maxLevel=3, criteria=term_crit)
# 루카스 카나데 opticalflow 함수에 넣는 파라미터

#3
t = 0
while True:


    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    print('t =', t)
    imgC = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    imgC = cv2.GaussianBlur(imgC, (5,5), 0.0)
    # 프레임 가져와서 gray는 흑백으로 변경 후 가우시안 블러 적용
    
    #3-1
    if mouse_status==2: # 마우스 드래그 중
        x1, y1, x2, y2 = roi
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
    
    #3-2
    if mouse_status==3: # 마우스 손 뗀 상태, roi 설정 완료
        print('initialize...')
        mouse_status = 0
        x1, y1, x2, y2 = roi       # 좌표에 따른 roi 설정
        roi_mask[:, :] = 0         # 원래 블랙 
        roi_mask[y1:y2, x1:x2] = 1 # roi 영역만 1로 셋팅
        p1 = cv2.goodFeaturesToTrack(imgC, mask=roi_mask, **params) # 피처를 찾기 위해 좋은 트랙킹을 찾는다, 최종 피처가 남는다
        if len(p1)>=4:             # 피처가 4개 이상이면
            p1 = cv2.cornerSubPix(imgC, p1, (5,5), (-1,-1), term_crit) # 코너를 찾는다
            rect = cv2.minAreaRect(p1) # 코너가 이루는 최소 사각형 구간
            box_pts = cv2.boxPoints(rect).reshape(-1,1,2) # 포인트로 변경 저장
            tracking_start = True # 트랙킹 시작
            
    #3-3
    if tracking_start: # 트랙킹 시작되었다면,
        p2, st, err = cv2.calcOpticalFlowPyrLK(imgP, imgC, p1, None, **params2) #옵티컬플로우 피라미드 LK
        p1r, st, err = cv2.calcOpticalFlowPyrLK(imgC, imgP, p2, None, **params2)
        # imgC는 가우시안 블러한 현재 프레임, imgP는 현재 프레임을 카피한 것. 반복문이므로 다음 루프 때 이전 루프의 imgC와 동일
        # 즉 imgC는 current frame, imgP는 previous frame
        # p2에 이전 프레임 ~ 현재 프레임 optical flow 저장, p1r에 현재 프레임 ~ 이전 프레임 optical flow 저장
        # 이유는 두 개의 차이가 적어야 optical flow에 문제가 없는 것
        d = abs(p1-p1r).reshape(-1,2).max(-1)
        # 그래서 두 차이를 계산
        stat = d < 1.0 # 1.0 is distance threshold, 차이 기준 1
        good_p2 = p2[stat==1].copy() # p2는 optical flow
        good_p1 = p1[stat==1].copy() # p1은 good feature
        for x, y in good_p2.reshape(-1, 2): # p2에 저장되어 있는 것들을 원으로 표시
            cv2.circle(frame, (int(x), int(y)), 3, (0,0,255), -1)
        
        if len(good_p2)<4: # 4보다 적으면 넘어감
            continue
        
        H, mask = cv2.findHomography(good_p1, good_p2, cv2.RANSAC, 3.0) # p1, p2로 네모 찾음 (매칭)
        box_pts = cv2.perspectiveTransform(box_pts, H) # 찾고자 하는 오브젝트의 바운딩 박스 생성
        cv2.polylines(frame, [np.int32(box_pts)], True, (255,0,0), 2) # 바운딩 박스 그림
        p1 = good_p2.reshape(-1,1,2)
        
    #3-4
    cv2.imshow('tracking', frame)
    imgP = imgC.copy()
    key = cv2.waitKey(25)
    if key == 27:
        break

if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

t = 1
t = 2
t = 3
t = 4
t = 5
t = 6
t = 7
t = 8
t = 9
t = 10
t = 11
t = 12
t = 13
t = 14
t = 15
t = 16
t = 17
t = 18
t = 19
t = 20
t = 21
t = 22
t = 23
t = 24
t = 25
t = 26
t = 27
t = 28
t = 29
t = 30
t = 31
t = 32
t = 33
t = 34
t = 35
t = 36
t = 37
t = 38
t = 39
t = 40
t = 41
t = 42
t = 43
t = 44
t = 45
t = 46
t = 47
t = 48
t = 49
t = 50
t = 51
t = 52
t = 53
t = 54
t = 55
t = 56
t = 57
t = 58
t = 59
t = 60
t = 61
t = 62
t = 63
t = 64
t = 65
t = 66
t = 67
t = 68
t = 69
t = 70
t = 71
t = 72
t = 73
t = 74
t = 75
t = 76
t = 77
t = 78
t = 79
t = 80
t = 81
t = 82
t = 83
t = 84
t = 85
t = 86
t = 87
t = 88
t = 89
t = 90
t = 91
t = 92
t = 93
t = 94
t = 95
t = 96
t = 97
t = 98
t = 99
t = 100
t = 101
t = 102
t = 103
t = 104
t = 105
t = 106
t = 107
t = 108
t = 109
t = 110
t = 111
t = 112
t = 113
t = 114
t = 115
t = 116
initialize...
t = 117
t = 118
t = 119
initialize...
t = 120
t = 121
t = 122
t = 123
t = 124
t = 125
t = 126
t = 127
t = 128
t = 129
t = 130
t = 131
t = 132
t = 133
t = 134
t = 135


-1

In [43]:
import cv2
import numpy as np

#1
roi = None
drag_start = None
mouse_status = 0
tracking_start = False

# 4개 변수 초기화, 마우스 이벤트에 사용

# mouse event 함수 정의
def onMouse(event, x, y, flags, param=None):
    global roi
    global drag_start
    global mouse_status
    global tracking_start
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drag_start = (x, y) # 마우스 드래그 포인트
        mouse_status = 1  # 1은 roi를 설정하기 시작했다는 의미
        tracking_start = False # 아직 트랙킹 안 한 상태
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            xmin = min(x, drag_start[0])
            ymin = min(y, drag_start[1])
            xmax = max(x, drag_start[0])
            ymax = max(y, drag_start[1])
            roi = (xmin, ymin, xmax, ymax)
            mouse_status = 2 # dragging
            # 마우스 시작포인트, 끝 포인트 즉 roi 영역 저장, 드래그 하는 중이라는 의미
            
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_status = 3 # complete 마우스에서 손을 떼었다라는 의미

#2
cv2.namedWindow('tracking') # tracking 이라는 윈도우 생성
cv2.setMouseCallback('tracking', onMouse) # onMous에 등록

path = './'
cap = cv2.VideoCapture('/Users/iseung-won/images/lucas2.mp4') # 새로운 영상 읽기
if (not cap.isOpened()):
    print('Error opening video')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), 
                 int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
roi_mask = np.zeros((height, width), dtype=np.uint8) 
# 동영상과 같은 사이즈의 roi 마스크 생성. black

params = dict(maxCorners=16, qualityLevel=0.1, minDistance=1, blockSize=5)
# 밑에 goodFeaturesToTrack에 사용
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
# 계속 돌아가지 않게 종료 조건 설정 시 사용, 10번
params2 = dict(winSize=(5,5), maxLevel=3, criteria=term_crit)
# 루카스 카나데 opticalflow 함수에 넣는 파라미터

#3
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    print('t =', t)
    imgC = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    imgC = cv2.GaussianBlur(imgC, (5,5), 0.0)
    # 프레임 가져와서 gray는 흑백으로 변경 후 가우시안 블러 적용
    
    #3-1
    if mouse_status==2: # 마우스 드래그 중
        x1, y1, x2, y2 = roi
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
    
    #3-2
    if mouse_status==3: # 마우스 손 뗀 상태, roi 설정 완료
        print('initialize...')
        mouse_status = 0
        x1, y1, x2, y2 = roi       # 좌표에 따른 roi 설정
        roi_mask[:, :] = 0         # 원래 블랙 
        roi_mask[y1:y2, x1:x2] = 1 # roi 영역만 1로 셋팅
        p1 = cv2.goodFeaturesToTrack(imgC, mask=roi_mask, **params) # 피처를 찾기 위해 좋은 트랙킹을 찾는다, 최종 피처가 남는다
        if len(p1)>=4:             # 피처가 4개 이상이면
            p1 = cv2.cornerSubPix(imgC, p1, (5,5), (-1,-1), term_crit) # 코너를 찾는다
            rect = cv2.minAreaRect(p1) # 코너가 이루는 최소 사각형 구간
            box_pts = cv2.boxPoints(rect).reshape(-1,1,2) # 포인트로 변경 저장
            tracking_start = True # 트랙킹 시작
            
    #3-3
    if tracking_start: # 트랙킹 시작되었다면,
        p2, st, err = cv2.calcOpticalFlowPyrLK(imgP, imgC, p1, None, **params2) #옵티컬플로우 피라미드 LK
        p1r, st, err = cv2.calcOpticalFlowPyrLK(imgC, imgP, p2, None, **params2)
        # imgC는 가우시안 블러한 현재 프레임, imgP는 현재 프레임을 카피한 것. 반복문이므로 다음 루프 때 이전 루프의 imgC와 동일
        # 즉 imgC는 current frame, imgP는 previous frame
        # p2에 이전 프레임 ~ 현재 프레임 optical flow 저장, p1r에 현재 프레임 ~ 이전 프레임 optical flow 저장
        # 이유는 두 개의 차이가 적어야 optical flow에 문제가 없는 것
        d = abs(p1-p1r).reshape(-1,2).max(-1)
        # 그래서 두 차이를 계산
        stat = d < 1.0 # 1.0 is distance threshold, 차이 기준 1
        good_p2 = p2[stat==1].copy() # p2는 optical flow
        good_p1 = p1[stat==1].copy() # p1은 good feature
        print('good_p1=',good_p1)
        print('good_p2=',good_p2)
        for x, y in good_p2.reshape(-1, 2): # p2에 저장되어 있는 것들을 원으로 표시
            cv2.circle(frame, (int(x), int(y)), 3, (0,0,255), -1)
        
        if len(good_p2)<4: # 4보다 적으면 넘어감
            continue
        
        H, mask = cv2.findHomography(good_p1, good_p2, cv2.RANSAC, 3.0) # p1, p2로 네모 찾음 (매칭)
        box_pts = cv2.perspectiveTransform(box_pts, H) # 찾고자 하는 오브젝트의 바운딩 박스 생성
        cv2.polylines(frame, [np.int32(box_pts)], True, (255,0,0), 2) # 바운딩 박스 그림
        p1 = good_p2.reshape(-1,1,2)
        
    #3-4
    cv2.imshow('tracking', frame)
    imgP = imgC.copy()
    key = cv2.waitKey(25)
    if key == 27:
        break

if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

t = 1
t = 2
t = 3
t = 4
t = 5
t = 6
t = 7
t = 8
t = 9
t = 10
t = 11
t = 12
t = 13
t = 14
t = 15
t = 16
t = 17
t = 18
t = 19
t = 20
t = 21
t = 22
t = 23
t = 24
t = 25
t = 26
t = 27
t = 28
t = 29
t = 30
t = 31
t = 32
t = 33
t = 34
t = 35
t = 36
t = 37
t = 38
t = 39
t = 40
t = 41
t = 42
t = 43
t = 44
t = 45
t = 46
t = 47
t = 48
t = 49
t = 50
t = 51
t = 52
t = 53
t = 54
t = 55
t = 56
t = 57
t = 58
t = 59
t = 60
t = 61
t = 62
t = 63
t = 64
t = 65
t = 66
t = 67
t = 68
t = 69
t = 70
t = 71
t = 72
t = 73
t = 74
t = 75
t = 76
t = 77
t = 78
t = 79
t = 80
t = 81
t = 82
t = 83
t = 84
t = 85
t = 86
t = 87
t = 88
t = 89
t = 90
t = 91
t = 92
t = 93
t = 94
t = 95
t = 96
t = 97
t = 98
initialize...
good_p1= [[[ 603.66815 1189.2666 ]]

 [[ 630.42584 1176.2803 ]]

 [[ 622.5323  1187.3119 ]]

 [[ 616.07635 1184.5674 ]]

 [[ 671.4014  1174.8217 ]]

 [[ 625.27893 1177.9714 ]]

 [[ 593.0361  1191.8489 ]]

 [[ 660.8266  1178.8245 ]]

 [[ 619.93475 1188.4235 ]]

 [[ 635.      1187.     ]]

 [[ 631.      1191. 

t = 107
good_p1= [[[ 597.88885 1184.1277 ]]

 [[ 624.6506  1171.2079 ]]

 [[ 616.82666 1182.117  ]]

 [[ 610.32336 1179.5602 ]]

 [[ 665.458   1169.7361 ]]

 [[ 619.4975  1172.9244 ]]

 [[ 587.43994 1186.7288 ]]

 [[ 654.9306  1173.7389 ]]

 [[ 614.1608  1183.2828 ]]

 [[ 629.2548  1181.8036 ]]

 [[ 625.2025  1185.7134 ]]

 [[ 662.3606  1152.2101 ]]

 [[ 652.1264  1157.1719 ]]

 [[ 639.10486 1165.4843 ]]

 [[ 646.1993  1172.8583 ]]

 [[ 634.119   1177.8082 ]]]
good_p2= [[[ 598.65625 1183.9586 ]]

 [[ 625.40186 1171.1919 ]]

 [[ 617.5491  1182.0731 ]]

 [[ 611.10065 1179.3384 ]]

 [[ 666.1527  1169.7074 ]]

 [[ 620.25433 1172.8605 ]]

 [[ 588.2498  1186.6274 ]]

 [[ 655.6519  1173.6238 ]]

 [[ 614.96655 1183.1725 ]]

 [[ 629.9826  1181.7626 ]]

 [[ 625.9309  1185.7344 ]]

 [[ 663.1248  1152.2058 ]]

 [[ 652.70215 1157.1516 ]]

 [[ 639.7812  1165.4636 ]]

 [[ 646.8162  1172.855  ]]

 [[ 634.7452  1177.7465 ]]]
t = 108
good_p1= [[[ 598.65625 1183.9586 ]]

 [[ 625.40186 1171.1919 ]]

 [[ 6

t = 117
good_p1= [[[ 604.17017 1183.8132 ]]

 [[ 630.45966 1171.1138 ]]

 [[ 622.7661  1181.885  ]]

 [[ 616.3877  1179.3494 ]]

 [[ 670.57544 1169.7224 ]]

 [[ 625.3546  1172.668  ]]

 [[ 593.82965 1186.187  ]]

 [[ 660.2337  1173.6068 ]]

 [[ 620.18964 1183.0035 ]]

 [[ 634.9526  1181.5245 ]]

 [[ 631.0222  1185.4867 ]]

 [[ 667.40704 1152.4072 ]]

 [[ 657.39545 1157.2582 ]]

 [[ 644.66724 1165.5105 ]]

 [[ 651.57684 1172.7618 ]]

 [[ 639.6464  1177.5834 ]]]
good_p2= [[[ 604.76575 1184.4161 ]]

 [[ 630.96954 1171.7128 ]]

 [[ 623.2666  1182.4553 ]]

 [[ 616.8487  1179.8121 ]]

 [[ 671.0304  1170.4545 ]]

 [[ 625.9154  1173.4438 ]]

 [[ 594.38226 1186.8899 ]]

 [[ 660.6605  1174.2313 ]]

 [[ 620.6787  1183.5295 ]]

 [[ 635.5046  1182.1824 ]]

 [[ 631.48816 1186.0067 ]]

 [[ 667.96246 1153.1321 ]]

 [[ 657.84534 1157.9883 ]]

 [[ 645.14307 1166.1399 ]]

 [[ 652.09875 1173.4716 ]]

 [[ 640.153   1178.2902 ]]]
t = 118
good_p1= [[[ 604.76575 1184.4161 ]]

 [[ 630.96954 1171.7128 ]]

 [[ 6

t = 127
good_p1= [[[ 608.15106 1186.7643 ]]

 [[ 626.2587  1184.8826 ]]

 [[ 620.0293  1182.3623 ]]

 [[ 672.99493 1173.0284 ]]

 [[ 628.694   1175.975  ]]

 [[ 598.01025 1189.0779 ]]

 [[ 662.8925  1176.9274 ]]

 [[ 623.7712  1185.8799 ]]

 [[ 638.2644  1184.6067 ]]

 [[ 634.28876 1188.2676 ]]

 [[ 670.0967  1156.2148 ]]

 [[ 660.1012  1160.946  ]]

 [[ 647.66626 1168.812  ]]

 [[ 654.47327 1176.0531 ]]

 [[ 642.78986 1180.7627 ]]]
good_p2= [[[ 607.55597 1188.1664 ]]

 [[ 625.62177 1186.2748 ]]

 [[ 619.4657  1183.7296 ]]

 [[ 672.3603  1174.5624 ]]

 [[ 628.225   1177.3162 ]]

 [[ 597.373   1190.488  ]]

 [[ 662.2573  1178.2767 ]]

 [[ 623.13947 1187.2849 ]]

 [[ 637.6469  1186.0144 ]]

 [[ 633.6569  1189.6161 ]]

 [[ 669.2754  1157.6725 ]]

 [[ 659.45746 1162.3751 ]]

 [[ 647.05    1170.2795 ]]

 [[ 653.8968  1177.434  ]]

 [[ 642.1831  1182.1952 ]]]
t = 128
good_p1= [[[ 607.55597 1188.1664 ]]

 [[ 625.62177 1186.2748 ]]

 [[ 619.4657  1183.7296 ]]

 [[ 672.3603  1174.5624 ]]

 [[ 6

t = 137
good_p1= [[[ 617.353   1203.3502 ]]

 [[ 635.33105 1201.5098 ]]

 [[ 629.1463  1198.9987 ]]

 [[ 681.8421  1189.9149 ]]

 [[ 637.922   1192.7037 ]]

 [[ 607.2419  1205.6133 ]]

 [[ 671.7322  1193.6143 ]]

 [[ 632.8257  1202.5385 ]]

 [[ 647.374   1201.3245 ]]

 [[ 643.36255 1204.953  ]]

 [[ 678.8116  1173.091  ]]

 [[ 669.0894  1177.7054 ]]

 [[ 656.64716 1185.6417 ]]

 [[ 663.52234 1192.7716 ]]

 [[ 651.733   1197.4628 ]]]
good_p2= [[[ 617.7798  1203.9823 ]]

 [[ 635.79175 1202.1766 ]]

 [[ 629.6839  1199.5555 ]]

 [[ 682.34076 1190.6604 ]]

 [[ 638.42474 1193.3776 ]]

 [[ 607.6907  1206.3407 ]]

 [[ 672.25836 1194.2987 ]]

 [[ 633.3937  1203.1527 ]]

 [[ 647.8455  1202.0455 ]]

 [[ 643.8718  1205.6597 ]]

 [[ 679.342   1173.818  ]]

 [[ 669.579   1178.4615 ]]

 [[ 657.1865  1186.3651 ]]

 [[ 664.09937 1193.5002 ]]

 [[ 652.2964  1198.2045 ]]]
t = 138
good_p1= [[[ 617.7798  1203.9823 ]]

 [[ 635.79175 1202.1766 ]]

 [[ 629.6839  1199.5555 ]]

 [[ 682.34076 1190.6604 ]]

 [[ 6

...

t = 812
good_p1= [[[ 640.9301  1213.411  ]]

 [[ 659.22424 1219.1515 ]]

 [[ 660.317   1224.9441 ]]

 [[ 666.22986 1216.5066 ]]

 [[ 659.4456  1217.8323 ]]]
good_p2= [[[ 641.6774  1213.4114 ]]

 [[ 659.69403 1218.8868 ]]

 [[ 660.7493  1224.7369 ]]

 [[ 666.66473 1216.3601 ]]

 [[ 659.9471  1217.6068 ]]]
t = 813
good_p1= [[[ 641.6774  1213.4114 ]]

 [[ 659.69403 1218.8868 ]]

 [[ 660.7493  1224.7369 ]]

 [[ 666.66473 1216.3601 ]]

 [[ 659.9471  1217.6068 ]]]
good_p2= [[[ 642.0797  1212.8275 ]]

 [[ 660.06256 1218.5269 ]]

 [[ 661.1616  1224.3187 ]]

 [[ 666.73456 1215.914  ]]

 [[ 660.2806  1217.2379 ]]]
t = 814
good_p1= [[[ 642.0797  1212.8275 ]]

 [[ 660.06256 1218.5269 ]]

 [[ 661.1616  1224.3187 ]]

 [[ 666.73456 1215.914  ]]

 [[ 660.2806  1217.2379 ]]]
good_p2= [[[ 642.91504 1212.3959 ]]

 [[ 661.13794 1218.2189 ]]

 [[ 662.3453  1223.8986 ]]

 [[ 667.71906 1215.4968 ]]

 [[ 661.3243  1216.8384 ]]]
t = 815
good_p1= [[[ 642.91504 1212.3959 ]]

 [[ 661.13794 1218.2189 ]]

 [[ 662.3

t = 842
good_p1= [[[ 618.7826  1241.1681 ]]

 [[ 617.2866  1246.4974 ]]

 [[ 632.1293  1232.9708 ]]

 [[ 619.23926 1238.2684 ]]]
good_p2= [[[ 616.3352  1241.6436 ]]

 [[ 614.7395  1246.9686 ]]

 [[ 629.8605  1233.2113 ]]

 [[ 616.78186 1238.6002 ]]]
t = 843
good_p1= [[[ 616.3352  1241.6436 ]]

 [[ 614.7395  1246.9686 ]]

 [[ 629.8605  1233.2113 ]]

 [[ 616.78186 1238.6002 ]]]
good_p2= [[[ 612.8809 1241.9396]]

 [[ 611.3169 1247.1908]]

 [[ 626.8085 1233.2772]]

 [[ 613.4765 1238.6497]]]
t = 844
good_p1= [[[ 612.8809 1241.9396]]

 [[ 611.3169 1247.1908]]

 [[ 626.8085 1233.2772]]

 [[ 613.4765 1238.6497]]]
good_p2= [[[ 610.5455  1241.2616 ]]

 [[ 608.8416  1246.2942 ]]

 [[ 624.67206 1232.1721 ]]

 [[ 611.0421  1237.6664 ]]]
t = 845
good_p1= [[[ 610.5455  1241.2616 ]]

 [[ 608.8416  1246.2942 ]]

 [[ 624.67206 1232.1721 ]]

 [[ 611.0421  1237.6664 ]]]
good_p2= [[[ 610.15015 1240.6366 ]]

 [[ 608.3801  1245.5503 ]]

 [[ 624.6503  1231.3085 ]]

 [[ 610.63727 1236.8145 ]]]
t = 846
good_p1=

good_p1= []
good_p2= []
t = 916
good_p1= []
good_p2= []
t = 917
good_p1= []
good_p2= []
t = 918
good_p1= []
good_p2= []
t = 919
good_p1= []
good_p2= []
t = 920
good_p1= []
good_p2= []
t = 921
good_p1= []
good_p2= []
t = 922
good_p1= []
good_p2= []
t = 923
good_p1= []
good_p2= []
t = 924
good_p1= []
good_p2= []
t = 925
good_p1= []
good_p2= []
t = 926
good_p1= [[[ 550.30927 1250.8104 ]]

 [[ 547.3188  1254.3508 ]]

 [[ 552.49274 1245.7832 ]]]
good_p2= [[[ 524.64606 1272.9294 ]]

 [[ 521.4962  1276.3872 ]]

 [[ 527.0442  1268.109  ]]]
t = 927
good_p1= [[[ 550.30927 1250.8104 ]]

 [[ 547.3188  1254.3508 ]]

 [[ 552.49274 1245.7832 ]]]
good_p2= [[[ 523.53503 1272.0984 ]]

 [[ 520.39215 1275.6633 ]]

 [[ 525.9036  1267.3247 ]]]
t = 928
good_p1= [[[ 550.30927 1250.8104 ]]

 [[ 552.49274 1245.7832 ]]]
good_p2= [[[ 522.4699 1270.4963]]

 [[ 524.6961 1265.7732]]]
t = 929
good_p1= [[[ 550.30927 1250.8104 ]]

 [[ 552.49274 1245.7832 ]]]
good_p2= [[[ 521.5176 1268.5201]]

 [[ 523.7648 1263.9609]]]


t = 1179
good_p1= []
good_p2= []
t = 1180
good_p1= []
good_p2= []
t = 1181
good_p1= []
good_p2= []
t = 1182
good_p1= []
good_p2= []
t = 1183
good_p1= []
good_p2= []
t = 1184
good_p1= []
good_p2= []
t = 1185
good_p1= []
good_p2= []
t = 1186
good_p1= []
good_p2= []
t = 1187
good_p1= []
good_p2= []
t = 1188
good_p1= []
good_p2= []
t = 1189
good_p1= []
good_p2= []
t = 1190
good_p1= []
good_p2= []
t = 1191
good_p1= []
good_p2= []
t = 1192
good_p1= []
good_p2= []
t = 1193
good_p1= []
good_p2= []
t = 1194
good_p1= []
good_p2= []
t = 1195
good_p1= []
good_p2= []
t = 1196
good_p1= []
good_p2= []
t = 1197
good_p1= []
good_p2= []
t = 1198
good_p1= []
good_p2= []
t = 1199
good_p1= []
good_p2= []
t = 1200
good_p1= []
good_p2= []
t = 1201
good_p1= []
good_p2= []
t = 1202
good_p1= []
good_p2= []
t = 1203
good_p1= []
good_p2= []
t = 1204
good_p1= []
good_p2= []
t = 1205
good_p1= []
good_p2= []
t = 1206
good_p1= []
good_p2= []
t = 1207
good_p1= []
good_p2= []
t = 1208
good_p1= []
good_p2= []
t = 1209
g

-1